In [35]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("CategoricalArrays")
Pkg.add("MLJ")
Pkg.add("MLJLinearModels")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [ ]:
#Se não estiver no ambiente de Julia para executar o notebook pode alterar manualmente ou rodar esse comando
using IJulia
notebook()

In [36]:
using CSV, DataFrames, CategoricalArrays, MLJ
@eval @load LogisticClassifier pkg=MLJLinearModels
@eval @load OneHotEncoder

import MLJLinearModels ✔
import MLJModels ✔


[ Info: For silent loading, specify `verbosity=0`. 
[ Info: For silent loading, specify `verbosity=0`. 


OneHotEncoder

In [37]:
train_df = CSV.read("/content/train.csv", DataFrame)
test_df = CSV.read("/content/test.csv", DataFrame)

Row,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
,Int64,Int64,String,String7,Float64?,Int64,Int64,String31,Float64?,String15?,String1
1,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,missing,Q
2,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0,missing,S
3,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,missing,Q
4,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,missing,S
5,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,missing,S
6,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.225,missing,S
7,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,missing,Q
8,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0,missing,S
9,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,missing,C


In [38]:
features = [:Pclass, :Sex, :Age, :SibSp, :Parch, :Fare]
train_df = dropmissing(select(train_df, [:Survived; features]))
test_df = select(test_df, features)

# Garantindo tipos corretos
train_df.Sex = categorical(train_df.Sex)
test_df.Sex = categorical(test_df.Sex)

418-element CategoricalArray{String7,1,UInt32}:
 String7("male")
 String7("female")
 String7("male")
 String7("male")
 String7("female")
 String7("male")
 String7("female")
 String7("male")
 String7("female")
 String7("male")
 String7("male")
 String7("male")
 String7("female")
 ⋮
 String7("male")
 String7("male")
 String7("female")
 String7("female")
 String7("female")
 String7("female")
 String7("female")
 String7("male")
 String7("female")
 String7("male")
 String7("male")
 String7("male")

In [39]:
X_train = select(train_df, features)
y_train = coerce(train_df.Survived, Multiclass)

714-element CategoricalArray{Int64,1,UInt32}:
 0
 1
 1
 1
 0
 0
 0
 1
 1
 1
 1
 0
 0
 ⋮
 0
 1
 1
 0
 0
 0
 0
 0
 0
 1
 1
 0

### Usando e treinando o modelo OneHotEncoder

In [40]:
hot = OneHotEncoder()
model = LogisticClassifier()
pipe = hot |> model

mach = machine(pipe, X_train, y_train)
fit!(mach)

[ Info: Training machine(ProbabilisticPipeline(one_hot_encoder = OneHotEncoder(features = Symbol[], …), …), …).
[ Info: Training machine(:one_hot_encoder, …).
[ Info: Spawning 2 sub-features to one-hot encode feature :Sex.
[ Info: Training machine(:logistic_classifier, …).
┌ Warning: f_tol is deprecated. Use f_abstol or f_reltol instead. The provided value (0.0001) will be used as f_reltol.
└ @ Optim ~/.julia/packages/Optim/8dE7C/src/types.jl:120
┌ Info: Solver: LBFGS{Optim.Options{Float64, Nothing}, @NamedTuple{}}
│   optim_options: Optim.Options{Float64, Nothing}
└   lbfgs_options: @NamedTuple{} NamedTuple()


trained Machine; does not cache data
  model: ProbabilisticPipeline(one_hot_encoder = OneHotEncoder(features = Symbol[], …), …)
  args: 
    1:	Source @982 ⏎ Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{Multiclass{2}}}}
    2:	Source @188 ⏎ AbstractVector{Multiclass{2}}


### Prevendo com o arquivo test.csv

In [45]:
# 1. Recarrega X_test com imputação
for col in [:Age, :Fare]
    test_df[!, col] = coalesce.(test_df[!, col], mean(skipmissing(test_df[!, col])))
end

test_df.Sex = categorical(test_df.Sex)
X_test = test_df

# 2. Predição
y_pred = predict_mode(mach, X_test)

418-element CategoricalArray{Int64,1,UInt32}:
 0
 0
 0
 0
 1
 0
 1
 0
 1
 0
 0
 0
 1
 ⋮
 0
 0
 1
 1
 1
 1
 1
 0
 1
 0
 0
 0

### Célular para gerar o CSV para envio no Kaggle

In [50]:
submission = DataFrame(
    PassengerId = test_ids,
    Survived = convert(Vector{Int}, y_pred)  # Converte de CategoricalVector{Int} para Vector{Int}
)
CSV.write("submission.csv", submission)

"submission.csv"